In [3]:
import os
import pandas as pd
import numpy as np
from scipy import misc
from sklearn.cross_validation import train_test_split
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [4]:
legend = pd.read_csv("/root/sharedfolder/legend/legend-clean.csv")

In [5]:
aug_path = "/root/sharedfolder/images-aug/"
legend_aug = pd.read_csv("/root/sharedfolder/legend/legend-aug.csv")

In [6]:
batch_size = 128
nb_classes = 8
nb_epoch = 12

# input image dimensions
img_rows, img_cols = 96, 96
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

In [9]:
legend['emotion'].unique()

array(['neutral', 'happiness', 'surprise', 'anger', 'sadness', 'contempt',
       'NEUTRAL', 'SADNESS', 'DISGUST', 'FEAR', 'SURPRISE', 'disgust',
       'ANGER', 'HAPPINESS', 'fear'], dtype=object)

In [10]:
legend['emotion'] = legend['emotion'].str.lower()

In [11]:
legend_aug['emotion'].unique()

array(['surprise', 'anger', 'sadness', 'contempt', 'disgust', 'fear'], dtype=object)

In [12]:
legend['emotion'].unique()

array(['neutral', 'happiness', 'surprise', 'anger', 'sadness', 'contempt',
       'disgust', 'fear'], dtype=object)

In [13]:
# 'neutral' = 0
# 'happiness' = 1
# 'surprise' = 2
# 'anger' = 3
# 'sadness' = 4
# 'contempt' = 5
# 'disgust' = 6
# 'fear' = 7
legend.replace('neutral', 0, inplace=True)
legend.replace('happiness', 1, inplace=True)
legend.replace('surprise', 2, inplace=True)
legend.replace('anger', 3, inplace=True)
legend.replace('sadness', 4, inplace=True)
legend.replace('contempt', 5, inplace=True)
legend.replace('disgust', 6, inplace=True)
legend.replace('fear', 7, inplace=True)

In [14]:
legend_aug.replace('surprise', 2, inplace=True)
legend_aug.replace('anger', 3, inplace=True)
legend_aug.replace('sadness', 4, inplace=True)
legend_aug.replace('contempt', 5, inplace=True)
legend_aug.replace('disgust', 6, inplace=True)
legend_aug.replace('fear', 7, inplace=True)

In [15]:
len(legend) + len(legend_aug)

36161

In [16]:
frames = [legend, legend_aug]
new_legend = pd.concat(frames)

In [17]:
len(new_legend)

36161

In [18]:
# convert images to numpy.ndarray

images = np.empty(shape=[len(new_legend), img_rows, img_cols])
for i in range(0,len(new_legend)):
    temp = aug_path + new_legend.iloc[i,0]
    img = misc.imread(temp, flatten=True,)
    img = misc.imresize(img, (img_rows, img_cols))
    images[i] = img

In [19]:
emotions = new_legend['emotion']

In [20]:
# the data, shuffled and split between train and test sets
X_train, X_test, y_train, y_test = train_test_split(images, emotions, test_size=0.25)

In [21]:
X_train.shape

(27120, 96, 96)

In [22]:
X_test.shape

(9041, 96, 96)

In [23]:
img_rows, img_cols = X_train.shape[1], X_train.shape[2]
X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)

In [24]:
X_train.shape

(27120, 1, 96, 96)

In [25]:
X_test.shape

(9041, 1, 96, 96)

In [26]:
input_shape = (1, img_rows, img_cols)

In [27]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

('X_train shape:', (27120, 1, 96, 96))
(27120, 'train samples')
(9041, 'test samples')
Train on 27120 samples, validate on 9041 samples
Epoch 1/12
27120/27120 [==============================] - 1751s - loss: 1.8562 - acc: 0.2692 - val_loss: 1.4302 - val_acc: 0.4737
Epoch 2/12
27120/27120 [==============================] - 1552s - loss: 1.4201 - acc: 0.4324 - val_loss: 1.1978 - val_acc: 0.5429
Epoch 3/12
27120/27120 [==============================] - 1591s - loss: 1.2281 - acc: 0.5290 - val_loss: 1.0631 - val_acc: 0.6016
Epoch 4/12
27120/27120 [==============================] - 1586s - loss: 1.0869 - acc: 0.5882 - val_loss: 0.9555 - val_acc: 0.6504
Epoch 5/12
27120/27120 [==============================] - 1599s - loss: 0.9732 - acc: 0.6385 - val_loss: 0.8885 - val_acc: 0.6810
Epoch 6/12
27120/27120 [==============================] - 1601s - loss: 0.8735 - acc: 0.6795 - val_loss: 0.8189 - val_acc: 0.7036
Epoch 7/12
27120/27120 [==============================] - 1550s - loss: 0.7887 - acc